In [2]:
import os
import random
import shutil
from PIL import Image

def embed_lsb(image: Image.Image, data_bytes: bytes) -> Image.Image:
    
    # Bettet die gegebenen data_bytes in die LSBs des 8-Bit-Grayscale-Bildes ein.
    # image: PIL Image im Modus 'L'
    # data_bytes: Zu embeddende Zufallsdaten (z.B. 98 Bytes für ein 28x28 Bild)
    
    if image.mode != 'L':
        raise ValueError("Bild muss im Modus L (8-Bit-Grayscale) sein.")
    
    width, height = image.size
    pixels = image.load()

    # Gesamtzahl Bits, die wir einbetten können
    max_bits = width * height  # 28*28 = 784 bei 28x28-Bildern
    
    # Gesamtzahl Bits der zu schreibenden Daten
    data_bits = len(data_bytes) * 8

    if data_bits > max_bits:
        raise ValueError("Zu viele Bytes für das verfügbare LSB-Volumen.")
    
    bit_index = 0
    for y in range(height):
        for x in range(width):
            if bit_index < data_bits:
                # Aktuelles Pixel auslesen
                px_val = pixels[x, y]
                # Byte und Bit position ermitteln
                byte_idx = bit_index // 8
                bit_pos = 7 - (bit_index % 8)   # von links nach rechts (MSB->LSB im Byte)
                
                # Bit aus dem Datenbyte extrahieren
                bit = (data_bytes[byte_idx] >> bit_pos) & 1
                
                # LSB des Pixelwertes auf dieses Bit setzen
                px_val_new = (px_val & 0xFE) | bit
                
                # Pixel aktualisieren
                pixels[x, y] = px_val_new
                bit_index += 1
            else:
                # Falls keine Daten mehr zu embedden sind, nichts mehr ändern
                break
    
    return image

def main(input_folder: str, output_folder_normal: str, output_folder_stego: str):
    
    # 1. Liest alle PNGs im input_folder ein.
    # 2. Mischt die Dateinamen zufällig.
    # 3. Teilt in zwei Hälften: eine Hälfte wird unverändert (normal), die andere mittels LSB-Stego verändert.
    # 4. Speichert die jeweiligen Bilder in output_folder_normal (unverändert) und output_folder_stego (verändert).
    
    
    # 1) Alle PNG-Dateien finden
    file_list = [f for f in os.listdir(input_folder) if f.lower().endswith(".png")]
    
    if not file_list:
        print("Keine PNG-Dateien im Ordner gefunden!")
        return
    
    # 2) Shuffle
    random.shuffle(file_list)
    
    # 3) Aufteilung in zwei Hälften
    half = len(file_list) // 2
    stego_files = file_list[:half]
    normal_files = file_list[half:]
    
    
    # 4) Verarbeitung der Bilder
    for idx, fname in enumerate(file_list):
        img_path = os.path.join(input_folder, fname)
        
        # Bild öffnen
        img = Image.open(img_path).convert('L')  # sicherstellen, dass es 'L' ist
        
        if fname in stego_files:
            # -- STEGO-Bild (positiv) --
            # Erzeuge zufällige 98 Bytes (784 Bits) – passend für 28x28 L-Bild
            # Für kleinere/nicht 28x28 solltest du das dynamisch berechnen
            random_bytes = bytes(random.getrandbits(8) for _ in range(98))
            
            # LSB-Embedding
            img_stego = embed_lsb(img, random_bytes)
            
            # Speichern im output_folder_stego
            out_path = os.path.join(output_folder_stego, fname)
            img_stego.save(out_path, format="PNG")
            
        else:
            # -- UNVERÄNDERTES Bild (negativ) --
            out_path = os.path.join(output_folder_normal, fname)
            # Einfach kopieren oder direkt abspeichern
            # Variante 1: Direkt kopieren
            # shutil.copy2(img_path, out_path)
            
            # Variante 2: Einmal "img" neu speichern
            img.save(out_path, format="PNG")
    
    print("Fertig! Es wurden {} Bilder steganographisch verändert und {} Bilder unverändert gespeichert.".format(
        len(stego_files), len(normal_files)
    ))


In [5]:
input_folder = "/Users/flaviohorak/Desktop/Bachelorarbeit/notebooks/createLSB/mnist_images"
output_folder_normal = "/Users/flaviohorak/Desktop/Bachelorarbeit/notebooks/createLSB/data/clean"
output_folder_stego = "/Users/flaviohorak/Desktop/Bachelorarbeit/notebooks/createLSB/data/stego"
    
main(input_folder, output_folder_normal, output_folder_stego)

Fertig! Es wurden 35000 Bilder steganographisch verändert und 35000 Bilder unverändert gespeichert.
